In [2]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

import module as module

In [3]:
#fichier données
data = './Data/Source/'
app_train = pd.read_csv(data+'application_train.csv')
app_test = pd.read_csv(data+'application_test.csv')

In [4]:
module.repartitionCibles(app_train)

Ratio des valeurs cibles
valeurs positives 8.1% (24825)
valeurs négatives 91.9% (282686)


In [132]:
X_train, X_test, labels, features = module.traitementDonnées(app_train, app_test, smote=False)

----------- Datas before traitments ----------
Ratio des valeurs cibles
valeurs positives 8.1% (24825)
valeurs négatives 91.9% (282686)
..............................................
Training data shape:  (307511, 130)
Testing data shape:  (48744, 129)
----------- Datas after traitments -----------
Ratio des valeurs cibles
valeurs positives 8.1% (24825)
valeurs négatives 91.9% (282686)
..............................................
Training data shape:  (307511, 260)
Testing data shape:  (48744, 260)
------------- Time of traitments -------------
Time: 10.8s


In [208]:
X_train, X_test, labels, features = module.traitementDonnées(app_train, app_test, smote=True)

----------- Datas before traitments ----------
Ratio des valeurs cibles
valeurs positives 8.1% (24825)
valeurs négatives 91.9% (282686)
..............................................
Training data shape:  (307511, 130)
Testing data shape:  (48744, 129)
------------- application SMOTE --------------


2025/01/02 19:25:31 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2025/01/02 19:25:31 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2025/01/02 19:25:32 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/01/02 19:25:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


----------- Datas after traitments -----------
Ratio des valeurs cibles
valeurs positives 50.0% (282686)
valeurs négatives 50.0% (282686)
..............................................
Training data shape:  (565372, 260)
Testing data shape:  (48744, 260)
------------- Time of traitments -------------
Time: 20.0s


In [182]:
from sklearn.linear_model import LogisticRegression
import mlflow
from mlflow.models import infer_signature
from sklearn.metrics import mean_squared_error

mlflow.set_tracking_uri(uri="http://10.0.50.72:5000")
mlflow.set_experiment(experiment_id="922754860405524204")
mlflow.set_tag('mlflow.runName', 'LogisticRegression')
mlflow.autolog()

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(X_train, labels)
log_reg_train = log_reg.predict_proba(X_train)[:, 1]

2025/01/02 18:36:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/01/02 18:36:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/01/02 18:36:51 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


In [184]:
log_reg_pred = log_reg.predict_proba(X_test)[:, 1]

In [188]:
# Infer the model signature
y_pred = log_reg.predict(X_train)
signature = mlflow.models.infer_signature(X_train, labels)

# Log parameters and metrics using the MLflow APIs
#mlflow.log_params(params)
mlflow.log_metrics({"mse": mean_squared_error(labels, y_pred)})

# Log the sklearn model and register as version 1
mlflow.sklearn.log_model(
    sk_model=log_reg,
    artifact_path="sklearn-model",
    signature=signature,
)

In [225]:
import importlib 
importlib.reload(module)


<module 'module' from 'Z:\\P7\\module.py'>